In [1]:
#get users that need to be scraped for images
import pandas as pd
import joblib
import numpy as np

In [4]:
df_de = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/german_data/data_for_models_german_data.pkl')

In [5]:
users_de = list(set(df_de.user_id.values.tolist()))

In [6]:
tw_de = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/german_data/german_tweets.pkl')

In [5]:
tw_de['date'] = pd.to_datetime(tw_de.created_at)
tw_de = tw_de[np.logical_not(tw_de.text.str.startswith('RT @'))].copy()
md_de = tw_de.groupby('user_id').agg({'date':'max'}).reset_index()
tw_de = tw_de.merge(md_de,on=['user_id','date'],how='inner')
tw_de = tw_de.loc[tw_de['user_id'].isin(users_de),'id'].values.tolist()
tw_de = [str(int(x)) for x in tw_de]

In [6]:
df_it = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/user_age_gender_location_train_set.pkl')

In [8]:
users_it = list(set(df_it.user_id.values.tolist()))

In [6]:
df_twt = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/tweets_by_user_id_clean.pkl')
df_twt['date'] = pd.to_datetime(df_twt.created_at)

In [7]:
df_twt = df_twt[np.logical_not(df_twt.RT)].copy()

In [8]:
md = df_twt.groupby('user_id').agg({'date':'max'}).reset_index()
tw_it = df_twt.merge(md,on=['user_id','date'],how='inner')
tw_it = tw_it['tweet_id'].values.tolist()
tw_it = [str(int(x)) for x in tw_it]

In [9]:
all_tweets = tw_it + tw_de

In [10]:
joblib.dump(all_tweets,"/g100_work/IscrC_mental/data/user_classification/images/users_wo_image.pkl")

['/g100_work/IscrC_mental/data/user_classification/images/users_wo_image.pkl']

## round 2 check the failed tweets and use the second newest in that case

In [2]:
#create a new list with users who had a failed tweet
tw_de = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/german_data/german_tweets.pkl')
tw_de['date'] = pd.to_datetime(tw_de.created_at)
tw_de = tw_de[np.logical_not(tw_de.text.str.startswith('RT @'))].copy()

In [3]:
failed = joblib.load("/g100_work/IscrC_mental/data/user_classification/images/failed.pkl")

In [4]:
failed_users = tw_de.loc[tw_de['id'].isin(failed),'user_id'].values.tolist()
df_failed = tw_de[tw_de['user_id'].isin(failed_users)]

In [5]:
md_f_de = df_failed.groupby('user_id').agg({'date':'max'}).reset_index()
df_failed = df_failed.merge(md_f_de,on=['user_id','date'],how='left',indicator=True)
df_failed = df_failed[df_failed['_merge']!='both']

In [6]:
md_f_de = df_failed.groupby('user_id').agg({'date':'max'}).reset_index()
df_failed = df_failed.merge(md_f_de,on=['user_id','date'],how='inner')
df_failed = df_failed['id'].values.tolist()
de_ids = [str(int(x)) for x in df_failed]

In [7]:
df_twt = pd.read_pickle('/g100_work/IscrC_mental/data/user_classification/tweets_by_user_id_clean.pkl')
df_twt['date'] = pd.to_datetime(df_twt.created_at)
df_twt = df_twt[np.logical_not(df_twt.RT)].copy()

In [8]:
failed_users = df_twt.loc[df_twt['tweet_id'].isin(failed),'user_id'].values.tolist()
df_failed = df_twt[df_twt['user_id'].isin(failed_users)]

In [9]:
md_f_it = df_failed.groupby('user_id').agg({'date':'max'}).reset_index()
df_failed = df_failed.merge(md_f_it,on=['user_id','date'],how='left',indicator=True)
df_failed = df_failed[df_failed['_merge']!='both']

In [10]:
md_f_it = df_failed.groupby('user_id').agg({'date':'max'}).reset_index()
df_failed = df_failed.merge(md_f_it,on=['user_id','date'],how='inner')
df_failed = df_failed['tweet_id'].values.tolist()
it_ids = [str(int(x)) for x in df_failed]

In [13]:
all_ids = it_ids + de_ids

In [15]:
joblib.dump(all_ids,"/g100_work/IscrC_mental/data/user_classification/images/users_wo_image_r2.pkl")

['/g100_work/IscrC_mental/data/user_classification/images/users_wo_image_r2.pkl']

## moving files

In [2]:
import os
import shutil
from tqdm import tqdm
import glob

In [3]:
base_path = '/g100_work/IscrC_mental/data/user_classification/images/'

In [12]:
for user in tqdm(users_it):
    try:
        src = os.path.join(base_path,f'new_scrape/{user}.jpg')
        dest = os.path.join(base_path,f'train/{user}.jpg')
        shutil.move(src,dest)
    except FileNotFoundError:
        pass

100%|██████████| 19200/19200 [00:36<00:00, 519.56it/s]


In [19]:
ni=glob.glob(f'{base_path}new_scrape/*')